In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
import urllib
import time
import csv
import os
import datetime

In [4]:
place=["台北一","台北二","板橋區","三重區","桃農","台中","豐原","東勢","嘉義","高雄","鳳山區","台東","宜蘭","南投"]
col_name=["date","crop_num","crop_name","market_num","market_name","high","medium","low","mean","volume"] 
crop_info=pd.read_csv("data/蔬果/crop_info.csv")
now = datetime.datetime.now()
today_date=now.year+"."+now.month+"."+now.day
for a in place:
    for b in range(crop_info.shape[0]):
        get_csv=pd.read_csv("data/蔬果/"+a+"/"+crop_info["Crop_name"][b]+".csv")
        csv_date=get_csv["date"][get_csv.shape[0]-1]
        temp=csv_date.split('-')
        last_date=temp[0]+"."+temp[1]+"."+temp[2]
        df=pd.DataFrame(columns=col_name)
        url="https://data.coa.gov.tw/Service/OpenData/FromM/FarmTransData.aspx?$top=1000&$skip=0"+"&Crop="+urllib.parse.quote(crop_info["Crop_name"][b])+"&StartDate=108.04.01&EndDate="+today_date+"&Market="+urllib.parse.quote(a)
        get = json.loads(urlopen(url).read()) 
        data = [] 
        for i in range(0,len(get)):
            if last_date==get[i]['交易日期']:
                break
            else:
                if get[i]['作物代號']==crop_info["Crop_num"][b]:
                    Date=get[i]['交易日期'].split('.')
                    year,month,date=Date[0],Date[1],Date[2]
                    year=str(int(year)+1911)
                    get[i]['交易日期']=year+'/'+month+'/'+date
                    data.append({"date":get[i]['交易日期'],"crop_num":get[i]['作物代號'],"crop_name":get[i]['作物名稱'],"market_num":get[i]['市場代號'],"market_name":get[i]['市場名稱'],"high":get[i]['上價'],"medium":get[i]['中價'],"low":get[i]['下價'],"mean":get[i]['平均價'],"volume":get[i]['交易量']}) 
        data.reverse() 
        df=pd.concat([pd.DataFrame(data), get_csv], ignore_index=True,sort=True)
        path="data/蔬果/"+a+"/"+crop_info["Crop_name"][b]+".csv"
        df["date"] = pd.to_datetime(df["date"])
        df["year"] = df["date"].dt.year
        df["month"] = df["date"].dt.month
        df["day"] = df["date"].dt.day
        df["dayofweek"] = df["date"].dt.dayofweek
        df.to_csv(path, encoding='utf_8_sig')
        time.sleep(3000.0/1000.0)

In [3]:
place_name=["雲林","嘉義","彰化","台南","高雄","屏東","台中","苗栗","桃園","台北"]
place_num=["C0K240","467480","C0G620","467410","467440","467590","467490","C0E420","C0C480","466920"]
date=[]
for year in ['2012','2013','2014','2015','2016','2017','2018']:
    for month in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']:
        date.append('-'.join([year,month]))
date.append('2019-01')
date.append('2019-02')
for a in range(0,len(place_name)):
    flag=0
    name=urllib.parse.quote(urllib.parse.quote(place_name[a]))
    for dd in date:
        url="http://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain"+"&station="+place_num[a]+"&stname="+name+"&datepicker="+str(dd)
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text)
        trs = soup.findAll('tr')
        ths = trs[2].findAll('th')
        day=1
        data=[]
        fdata=[]
        if flag==0:
            title=[]
            for th in ths:
                title.append(th.text)
            title.pop(0)
            title.append("year")
            title.append("month")
            title.append("day")
            df=pd.DataFrame(columns=title)
            flag=1
        for tr in trs[4:]:
            t=tr.findAll('td')
            for x in t:
                data.append(x.text.strip())
            sp=dd.split('-')
            data.pop(0)
            data.append(sp[0])
            data.append(sp[1])
            data.append(str(day))
            fdata.append({"測站氣壓(hPa)":data[0],"海平面氣壓(hPa)":data[1],"測站最高氣壓(hPa)":data[2],"測站最高氣壓時間(LST)":data[3],"測站最低氣壓(hPa)":data[4],"測站最低氣壓時間(LST)":data[5],"氣溫(℃)":data[6],"最高氣溫(℃)":data[7],"最高氣溫時間(LST)":data[8],"最低氣溫(℃)":data[9],"最低氣溫時間(LST)":data[10],"露點溫度(℃)":data[11],"相對溼度(%)":data[12],"最小相對溼度(%)":data[13],"最小相對溼度時間(LST)":data[14],"風速(m/s)":data[15],"風向(360degree)":data[16],"最大陣風(m/s)":data[17],"最大陣風風向(360degree)":data[18],"最大陣風風速時間(LST)":data[19],"降水量(mm)":data[20],"降水時數(hr)":data[21],"10分鐘最大降水量(mm)":data[22],"10分鐘最大降水起始時間(LST)":data[23]," 一小時最大降水量(mm)":data[24],"一小時最大降水量起始時間(LST)":data[25],"日照時數(hr)":data[26],"日照率(%)":data[27],"全天空日射量(MJ/㎡)":data[28],"能見度(km)":data[29],"A型蒸發量(mm)":data[30],"日最高紫外線指數":data[31],"日最高紫外線指數時間(LST)":data[32],"總雲量(0~10)":data[33],"year":data[34],"month":data[35],"day":data[36]})
            day=day+1
            data=[]
        df=pd.concat([df,pd.DataFrame(fdata)], ignore_index=True,sort=True)
    path=r"C:\Users\admin\Desktop\Project\test"+"\\"+place_name[a]+".csv"
    df.to_csv(path, encoding='utf_8_sig')

C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\admin\Anaconda3\envs\tensorflow\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [9]:
import datetime
now = datetime.datetime.now()
print("https://data.coa.gov.tw/Service/OpenData/FromM/FarmTransData.aspx?$top=1000&$skip=0&Crop="+urllib.parse.quote('朴菜')+"&StartDate=108.01.01&EndDate="+str(now.year-1911)+"."+now.strftime("%m.%d")+"&Market="+urllib.parse.quote('台北一'))

In [5]:
crop_info=pd.read_csv(r"C:\Users\admin\Desktop\crop_info.csv")
print(crop_info)

    Crop_name Crop_num Crop_type
0          香蕉       A1        香蕉
1     鳳梨-金鑽鳳梨       B2        鳳梨
2      小番茄-聖女       72       小番茄
3     木瓜-網室紅肉       I1        木瓜
4      火龍果-紅肉      812       火龍果
5       甘蔗-帶皮       61        甘蔗
6     百香果-改良種       51       百香果
7      西瓜-大西瓜       T1        西瓜
8      芒果-芒果青       R8        芒果
9       柿子-甜柿       Z4        柿子
10   洋香瓜-網狀紅肉       W1       洋香瓜
11     桃子-水蜜桃       Y1        桃子
12         草莓       45        草莓
13      荔枝-黑葉       J2        荔枝
14      梨-新興梨       O4         梨
15      甜瓜-美濃       V1        甜瓜
16      甜橙-柳橙       E1        甜橙
17     蛋黃果-仙桃       G1       蛋黃果
18         棗子       22        棗子
19         椪柑       C1        椪柑
20    番石榴-珍珠芭       P1       番石榴
21         椰子       11        椰子
22      楊桃-紅龍       M2        楊桃
23      葡萄-巨峰       S1        葡萄
24         酪梨       G3        酪梨
25     蓮霧-紅蓮霧       Q1        蓮霧
26   龍眼-龍眼乾帶殼       K4        龍眼
27         橄欖       G7        橄欖
28      雜柑-檸檬       F1        雜柑
29      蘋果

In [4]:
crop_info["Crop_name"][0]

'香蕉'

In [3]:
place_name=["雲林","嘉義","彰化","台南","高雄","屏東","台中","苗栗","桃園","台北","新北","基隆","新竹","南投","宜蘭","花蓮","台東"]
place_num=["C0K240","467480","C0G620","467410","467440","467590","467490","C0E420","C0C480","466920","466880","466940","C0D570","467650","467060","466990","467540"]
date=[]
for year in ['2012','2013','2014','2015','2016','2017','2018']:
    for month in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']:
        date.append('-'.join([year,month]))
date.append('2019-01')
date.append('2019-02')
date.append('2019-03')
date.append('2019-04')
for a in range(0,len(place_name)):
    flag=0
    name=urllib.parse.quote(urllib.parse.quote(place_name[a]))
    for dd in date:
        url="http://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain"+"&station="+place_num[a]+"&stname="+name+"&datepicker="+str(dd)
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text)
        trs = soup.findAll('tr')
        ths = trs[2].findAll('th')
        day=1
        data=[]
        fdata=[]
        if flag==0:
            title=[]
            for th in ths:
                title.append(th.text)
            title.pop(0)
            title.append("year")
            title.append("month")
            title.append("day")
            title.append("date")
            df=pd.DataFrame(columns=title)
            flag=1
        for tr in trs[4:]:
            t=tr.findAll('td')
            for x in t:
                data.append(x.text.strip())
            sp=dd.split('-')
            data.pop(0)
            data.append(sp[0])
            data.append(sp[1])
            data.append(str(day))
            data.append(sp[0]+'/'+sp[1]+'/'+str(day))
            fdata.append({"測站氣壓(hPa)":data[0],"海平面氣壓(hPa)":data[1],"測站最高氣壓(hPa)":data[2],"測站最高氣壓時間(LST)":data[3],"測站最低氣壓(hPa)":data[4],"測站最低氣壓時間(LST)":data[5],"氣溫(℃)":data[6],"最高氣溫(℃)":data[7],"最高氣溫時間(LST)":data[8],"最低氣溫(℃)":data[9],"最低氣溫時間(LST)":data[10],"露點溫度(℃)":data[11],"相對溼度(%)":data[12],"最小相對溼度(%)":data[13],"最小相對溼度時間(LST)":data[14],"風速(m/s)":data[15],"風向(360degree)":data[16],"最大陣風(m/s)":data[17],"最大陣風風向(360degree)":data[18],"最大陣風風速時間(LST)":data[19],"降水量(mm)":data[20],"降水時數(hr)":data[21],"10分鐘最大降水量(mm)":data[22],"10分鐘最大降水起始時間(LST)":data[23]," 一小時最大降水量(mm)":data[24],"一小時最大降水量起始時間(LST)":data[25],"日照時數(hr)":data[26],"日照率(%)":data[27],"全天空日射量(MJ/㎡)":data[28],"能見度(km)":data[29],"A型蒸發量(mm)":data[30],"日最高紫外線指數":data[31],"日最高紫外線指數時間(LST)":data[32],"總雲量(0~10)":data[33],"year":data[34],"month":data[35],"day":data[36],"date":data[37]})
            day=day+1
            data=[]
        df=pd.concat([df,pd.DataFrame(fdata)], ignore_index=True,sort=True)
        df["date"] = pd.to_datetime(df["date"])
        time.sleep(2000.0/1000.0)
    path=r"C:\Users\admin\Desktop\Project\train_data"+"\\"+place_name[a]+".csv"
    df.to_csv(path, encoding='utf_8_sig')
    time.sleep(4000.0/1000.0)

C:\Users\admin\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\admin\Anaconda3\envs\tensorflow\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [7]:
import pandas as pd
get_csv=pd.read_csv(r"C:\Users\admin\Desktop\train\data\蔬果\\"+"台北一"+"\\"+"香蕉"+".csv")
csv_date=get_csv["date"][get_csv.shape[0]-1]
print(csv_date)

2019-04-30


In [15]:
import pandas as pd
get_csv=pd.read_csv(r"C:\Users\admin\Desktop\train\data\蔬果\\"+"台北一"+"\\"+"香蕉"+".csv")
get_csv=get_csv.drop(["Unnamed: 0"], axis=1)
new_data = pd.DataFrame(get_csv[-1:].values, index=[get_csv.shape[0]], columns=get_csv.columns)
new_data["date"]="test"
get_csv=get_csv.append(new_data)
print(get_csv)

     crop_name crop_num        date  high   low market_name market_num  mean  \
0           香蕉       A1  2012-01-01    23  12.8         台北一        109  18.2   
1           香蕉       A1  2012-01-03  24.3  12.8         台北一        109  18.4   
2           香蕉       A1  2012-01-04  23.7    14         台北一        109  18.9   
3           香蕉       A1  2012-01-05  23.6  13.1         台北一        109  18.8   
4           香蕉       A1  2012-01-06  23.4  15.1         台北一        109  19.5   
5           香蕉       A1  2012-01-07  23.2  15.6         台北一        109  19.3   
6           香蕉       A1  2012-01-08    25  15.8         台北一        109  20.3   
7           香蕉       A1  2012-01-10  24.3  15.8         台北一        109  20.2   
8           香蕉       A1  2012-01-11    24  13.5         台北一        109  18.6   
9           香蕉       A1  2012-01-12    23    13         台北一        109  18.2   
10          香蕉       A1  2012-01-13    24    11         台北一        109  16.8   
11          香蕉       A1  2012-01-14  21.

In [22]:
import pandas as pd
import datetime

data=[]
get_year=str(now.year)
get_month=str(now.month)
get_day=str(now.day)
if int(now.month)<10:
    get_month="0"+get_month
if int(now.day)<10:
    get_day="0"+get_day
data.append({"date":get_year+"-"+get_month+"-"+get_day})
date_csv=pd.concat([date_csv,pd.DataFrame(data)], ignore_index=True,sort=True)

print(date_csv)

            date
0     2012-01-01
1     2012-01-02
2     2012-01-03
3     2012-01-04
4     2012-01-05
5     2012-01-06
6     2012-01-07
7     2012-01-08
8     2012-01-09
9     2012-01-10
10    2012-01-11
11    2012-01-12
12    2012-01-13
13    2012-01-14
14    2012-01-15
15    2012-01-16
16    2012-01-17
17    2012-01-18
18    2012-01-19
19    2012-01-20
20    2012-01-21
21    2012-01-22
22    2012-01-23
23    2012-01-24
24    2012-01-25
25    2012-01-26
26    2012-01-27
27    2012-01-28
28    2012-01-29
29    2012-01-30
...          ...
2648  2019-04-02
2649  2019-04-03
2650  2019-04-04
2651  2019-04-05
2652  2019-04-06
2653  2019-04-07
2654  2019-04-08
2655  2019-04-09
2656  2019-04-10
2657  2019-04-11
2658  2019-04-12
2659  2019-04-13
2660  2019-04-14
2661  2019-04-15
2662  2019-04-16
2663  2019-04-17
2664  2019-04-18
2665  2019-04-19
2666  2019-04-20
2667  2019-04-21
2668  2019-04-22
2669  2019-04-23
2670  2019-04-24
2671  2019-04-25
2672  2019-04-26
2673  2019-04-27
2674  2019-04-